# <center >OpenAI Agents SDK + MCP 智能体开发实战</center>

## <center>Part 3.  SSE & StreamableHttp MCP 应用开发实战</center>

&emsp;&emsp;本节课我们进一步补充`OpenAI`的`Agents SDK`框架接入`SSE`与`StreamableHttp`这两种更贴近企业应用的模式的`MCP`服务器，并且重点介绍`input_guardrail`输入护栏的实现原理以及在`Agents SDK`框架中的应用开发技巧。

&emsp;&emsp;首先，构建`Agent`组件对象时可用的参数配置如下所示：（绿色为课程中已经重点介绍过的参数）

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Agent 组件核心参数</font></p>
<div class="center">

| 属性名                | 类型                                                                                          | 描述                                                                                                   |
|---------------------|-----------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------|
| <font color=green>**name**</font>             | `str`                                                                                         | 代理的名称。                                                                                             |
| <font color=green>**instructions**</font>      | `str` \| `Callable[[RunContextWrapper[TContext], Agent[TContext]], MaybeAwaitable[str]]` \| `None` | 代理的指令，用作“系统提示”。可以是字符串或动态生成指令的函数。                                           |
| <font color=green>**handoff_description**</font> | `str` \| `None`                                                                              | 代理的描述，用于代理作为交接时，让 LLM 知道它的功能和何时调用它。                                         |
| <font color=green>**handoffs**</font>         | list[Agent[Any] | Handoff[TContext]]                                                   | 代理可以委托的子代理列表。允许关注点分离和模块化。                                                       |
| <font color=green>**model**</font>              | `str` \| `Model` \| `None`                                                                   | 调用 LLM 时使用的模型实现。默认情况下，如果未设置，代理将使用 `openai_provider.DEFAULT_MODEL` 中配置的默认模型。 |
| <font color=green>**model_settings**</font>    | `ModelSettings`                                                                               | 配置模型特定的调优参数（例如温度、top_p）。                                                             |
| <font color=green>**tools**</font>       | `list[Tool]`                                                                                 | 代理可以使用的工具列表。                                                                                 |
| <font color=green>mcp_servers</font>        | `list[MCPServer]`                                                                             | 代理可以使用的模型上下文协议（MCP）服务器列表。                                                        |
| <font color=green>mcp_config</font>        | `MCPConfig`                                                                                   | MCP 服务器的配置。                                                                                       |
| `input_guardrails`  | `list[InputGuardrail[TContext]]`                                                             | 在代理执行之前并行运行的检查列表，仅在代理是链中的第一个代理时运行。                                     |
| `output_guardrails` | `list[OutputGuardrail[TContext]]`                                                            | 在生成响应后对代理的最终输出运行的检查列表，仅在代理生成最终输出时运行。                                 |
| `output_type`       | type[Any] | AgentOutputSchemaBase | None                                                 | 输出对象的类型。如果未提供，输出将为 `str`。                                                             |
| <font color=green>hooks</font>            | AgentHooks[TContext] | None                                                               | 接收代理生命周期事件回调的类。                                                                           |
| `tool_use_behavior` | Literal["run_llm_again", "stop_on_first_tool"] | StopAtTools | ToolsToFinalOutputFunction | 配置工具使用的处理方式。                                                                                 |
| `reset_tool_choice` | `bool`                                                                                       | 调用工具后是否将工具选择重置为默认值。默认为 `True`。确保代理不会进入工具使用的无限循环。                   |

</div>

&emsp;&emsp;同时，对应的`Runner`组件对象时可用的参数配置如下所示：（绿色为课程中已经重点介绍过的参数）

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Runner 组件核心参数</font></p>
<div class="center">


| 属性名                     | 描述                                                                                                   |
|--------------------------|--------------------------------------------------------------------------------------------------------|
| <font color=green>**starting_agent**</font>        | 要运行的起始代理。                                                                                        |
|  <font color=green>**input**</font>                     | 代理的初始输入。可以传递一个用户消息的字符串，或一个输入项的列表。                                           |
| <font color=green>**context**<font color=red>                | 运行代理时使用的上下文。                                                                                 |
| `max_turns`             | 运行代理的最大回合数。回合定义为一次 AI 调用（包括可能发生的任何工具调用）。                                   |
| <font color=green>**hooks**</font>                  | 接收各种生命周期事件回调的对象。                                                                         |
| `run_config`             | 整个代理运行的全局设置。                                                                                 |
| `previous_response_id`   | 上一个响应的 ID，如果使用 OpenAI 模型通过 Responses API，这允许你跳过传递上一个回合的输入。                   |
</div>


# 一、SSE 模式MCP服务器接入实战

&emsp;&emsp;`OpenAI Agents SDK`框架除了支持`Stdio`通信协议的`MCP Server`外，还支持`SSE` 内置协议类型的`MCP Server`。 `SSE`的全称是`Server-Sent Events`，它是一种基于`HTTP`的轻量级协议，用于服务器向浏览器推送事件。`SSE` 模式下，`MCP Server` 会监听一个 `HTTP` 端口，并使用 `SSE` 协议发送事件消息。

&emsp;&emsp;在`OpenAI Agents SDK`框架中，`SSE` 模式下的`MCP Server` 需要使用`MCPServerSSE`类来创建。其源码定义：https://github.com/openai/openai-agents-python/blob/main/src/agents/mcp/server.py#L57


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505201033232.png" width=60%></div>

&emsp;&emsp;因为传输协议不同，所以在应用上所需要传递的参数也有所不同。与`Stdio`类似，`MCPServerSse`通过接收`MCPServerSseParams`中定义的参数来连接`MCP Server`。根据其源码定义，我们首先可以梳理出`MCPServerStdio`类能够接收的参数如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>MCPServerSseParams参数汇总</font></p>
<div class="center">

| 参数                     | 类型               | 描述                                                         | 使用场景                                                   |
|--------------------------|--------------------|--------------------------------------------------------------|----------------------------------------------------------|
| `url`                    | `str`              | 服务器的 URL。                                             | 指定要连接的服务器地址。                                 |
| `headers`                | `dict` (可选)      | 发送到服务器的请求头。                                     | 用于传递认证信息或其他自定义头部。                       |
| `timeout`                | `float` (可选)     | HTTP 请求的超时时间，默认为 5 秒。                         | 控制请求的最大等待时间，避免长时间无响应。               |
| `sse_read_timeout`       | `float` (可选)     | SSE 连接的超时时间，默认为 5 分钟。                        | 控制服务器推送事件的最大等待时间，确保连接的稳定性。     |   
<div>                

&emsp;&emsp;通过参数能明显感觉到，`Stdio` 模式把 `MCP` 服务器视作一个“要启动的本地进程”，因此最重要的参数是「怎么启动」(command、args、env、cwd… )。而`SSE` 模式把 `MCP` 服务器当作“已运行的 HTTP 服务”，所以它关心的是「怎么连过去」(url、headers、timeout… )。两套参数完全不同，也反映了各自的侧重点：`Stdio` 追求本地零网络成本与进程隔离；`SSE` 追求跨进程／跨机器的流式推送与共享复用。

&emsp;&emsp;其中，`timeout` 只管 `HTTP` 握手阶段，即如果在规定秒数内连不上 `SSE` 端点或拿不到首字节，就直接放弃。而`sse_read_timeout` 只管 `SSE` 流阶段：即连接一旦建立，就允许长时间保持打开，但如果在设定的间隔内 1 个字节都收不到，就判断连接已失活并自动重连。

&emsp;&emsp;同时，`MCPServerSse` 在使用上相较于`MCPServerStdio` 更为简单，只需要传递`url` 参数即可，而省去了相对复杂的本地虚拟运行环境的配置环节。

&emsp;&emsp;下面我们就通过实战案例来演示在`OpenAI Agents SDK`框架下`SSE` 模式下的`MCP` 服务器接入方法。

&emsp;&emsp;在 `SSE (Server-Sent Events)` 模式 下，我们可以把 `MCP` 服务器当成一个已经在网络上跑着的「工具 API」，用 `URL` 建立事件流、再用 `POST /messages` 发指令。当前主流的使用形式可以归纳为两大类：<font color="red">**自托管启动与平台托管直连。**</font>

- **自托管启动（Self-hosted）**

&emsp;&emsp;这种方式主要是应用[`MCP`官方的Python / TypeScript MCP SDK](https://github.com/modelcontextprotocol) 把 `Server` 代码跑起来（FastAPI / Express 集成）, 程序启动后会打印或返回一个本地或内网 `SSE URL`，然后便可以用这个 `URL` 建立事件流接入到`OpenAI Agents SDK` 框架中。如果需要使用`MCP` 官方的`Python` SDK构建`SSE` 模式的`MCP` 服务器，可以学习《【加餐】MCP SSE与流式HTTP开发实战】》课程模块，我们这里不再重复讲解。

- **平台托管直连（Platform-hosted）**

&emsp;&emsp;诸如 [`ModelScope`的 `MCP` 广场](https://www.modelscope.cn/mcp)、[mcp.run](https://docs.mcp.run/integrating/tutorials/mcp-run-sse-openai-agents)等社区维护大量 `MPC` 服务并支持一键启动`MCP`的`SSE` 模式。使用的方法非常简单，只需要在平台首页选好`MCP Server` → 生成一个`API-Key`，一键生成 `SSE URL` ，即可直接接入使用。

&emsp;&emsp;对于国内开发者，通过“一键复制 URL + API-Key”就能拿到 `SSE` 端点的热门网站大致可以分为四类：
1. 专门的 MCP 服务市场/目录（如 MCP Market、AIbase、MCP Server Hub）；
2. 云厂商推出的 官方托管平台（阿里云 “百炼” 与 Higress Marketplace）；
3. 行业 API 已内置 MCP & SSE（百度地图、腾讯位置服务等）；
4. IDE / 客户端集市 把第三方 SSE URL 聚合到本地（Cherry Studio、Cursor）。

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>MCP 热门导航网站</font></p>
<div class="center">

| 平台                            | 规模与特点                                                                      | 访问链接            |
| ----------------------------- | -------------------------------------------------------------------------- | ---------------- |
| **ModelScope MCP 广场** | 超过 3000+ 个中文托管 MCP；详情页右侧直接复制 SSE URL，部分服务需在“API Key”栏目填 Token | [点击进入](https://www.modelscope.cn/mcp/) |
| **MCP Market** | 标榜“国内首个 MCP 服务市场”，已收录 **1W+** SSE Server，并提供云托管与 Playground，一键生成专属 URL | [点击进入](https://mcpmarket.cn/) |
| **AIbase MCP 资源站**            | 整理 GitHub 上的热门 MCP Server 并直接给出 SSE 地址与使用教程                  | [点击进入](https://mcp.aibase.cn/)   |
| **MCP Server Hub**            | 国际化，标签筛选 + “Copy URL” 操作流畅，站内统计数千条 SSE Server                    | [点击进入](https://mcpserverhub.com/)]       | |
| **阿里云 百炼**                  | 官方托管“全周期 MCP 服务”，控制台能生成已鉴权的 SSE URL              | [点击进入](https://bailian.console.aliyun.com/?spm=5176.29619931.0.0.1369521cW5hVGj&tab=mcp#/mcp-market) |


&emsp;&emsp;平台托管直连不需要开发者关心进程与依赖等问题，直接在网页一键生成`URL`即可接入。下面我们以`ModelScope`的`MCP` 广场为例，演示如何在`OpenAI Agents SDK`框架中通过`SSE` 模式接入`MCP` 服务器。

- **Step 1. 进入`ModelScope`的`MCP` 广场，访问地址：https://www.modelscope.cn/mcp**

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505201154985.png" width=60%></div>

- **Step 2. 点击右上角`登录`按钮，登录`ModelScope`账号**

&emsp;&emsp;可以通过手机号注册并登录，注意：国内用户记得选择中国大陆 +86 的手机号注册，否则会提示手机号格式错误。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505201207597.png" width=60%></div>

- **Step 3. 挑选`MCP` 服务，并进入详情页**

&emsp;&emsp;这里我们选择的是`12306-MCP车票查询工具`，该`MCP` 服务提供了搜索12306购票信息的功能。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505201207598.png" width=60%></div>

- **Step 4. 点击`立即使用`按钮，进入`MCP` 服务详情页**

&emsp;&emsp;如果是首次使用，在详情页的最右侧会有一个`通过SSE URL`连接服务的说明

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505201207599.png" width=60%></div>

&emsp;&emsp;点击连接按钮，即可生成一个SSE URL，如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505201207600.png" width=60%></div>

- **Step 5. 复制SSE URL，并将其接入到`OpenAI Agents SDK`框架中**

&emsp;&emsp;在`OpenAI Agents SDK`框架中，可以通过`MCPServerSSE`类来创建`SSE` 模式的`MCP` 服务器。该类可接收的配置参数与`MCPServerStdio`类无异，只不过在`params`参数中需要接收的是`MCPServerSseParams`，而不再是`MCPServerStdioParams`。如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>MCPServerSse可接收参数</font></p>
<div class="center">



| 参数名称                          | 类型                          | 默认值 | 描述                                                                                                                                                                                                 |
|-----------------------------------|-------------------------------|--------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `params`                          | `MCPServerSseParams`        | -      | 配置服务器的参数，包括服务器的URL、请求头、HTTP请求超时和SSE连时。                                                               |
| `cache_tools_list`               | `bool`                        | `False`| 是否缓存工具列表。如果为 `True`，工具列表将被缓存，仅在第一次从服务器获取。如果为 `False`，每次调用 `list_tools()` 时都会从服务器获取工具列表。缓存可以通过调用 `invalidate_tools_cache()` 来失效。 |
| `name`                            | str 或 None               | `None` | 服务器的可读名称。如果未提供，将根据命令自动创建名称。                                                                                                                                              |
| `client_session_timeout_seconds`  | float 或 None                | `5`    | 传递给 MCP `ClientSession` 的读取超时时间时。                                                                                                                                                             |

&emsp;&emsp;`params`参数可接收的数据类型为`MCPServerSseParams`，如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>MCPServerSseParams参数汇总</font></p>
<div class="center">

| 参数                     | 类型               | 描述                                                         | 使用场景                                                   |
|--------------------------|--------------------|--------------------------------------------------------------|----------------------------------------------------------|
| `url`                    | `str`              | 服务器的 URL。                                             | 指定要连接的服务器地址。                                 |
| `headers`                | `dict` (可选)      | 发送到服务器的请求头。                                     | 用于传递认证信息或其他自定义头部。                       |
| `timeout`                | `float` (可选)     | HTTP 请求的超时时间，默认为 5 秒。                         | 控制请求的最大等待时间，避免长时间无响应。               |
| `sse_read_timeout`       | `float` (可选)     | SSE 连接的超时时间，默认为 5 分钟。                        | 控制服务器推送事件的最大等待时间，确保连接的稳定性。     |   
<div>                

- **Step 6. 创建`MCPServerSSE` 实例**

&emsp;&emsp;了解了具体的使用参数后，我们就可以创建`MCPServerSSE` 实例了。如下代码所示，可以通过`MCPServerSse`类中的静态方法`name`查看当前连接的`MCP` 服务器名称，同时也可以通过其父类`_MCPServerWithClientSession`中的`list_tools`方法查看当前连接的`MCP` 服务器提供的工具列表。

In [ ]:
from agents.mcp import MCPServerSse

async def main():
    async with MCPServerSse(
        name="12306_mcp",
        params={
            "url": "https://mcp.api-inference.modelscope.cn/sse/56a8f2b1578f44",  # 这里需要替换成 MCP的 SSE URL
            "timeout": 30,            # HTTP请求超时时间30秒
            "sse_read_timeout": 5   # 连接超时时间到5分钟
        },
        client_session_timeout_seconds=60,  # 增加客户端会话超时到60秒
        cache_tools_list=True,             # 启用工具列表缓存
    ) as mcp:
        print(f"连接MCP服务器成功: {mcp.name}")  # 通过静态方法name查看当前连接的MCP服务器名称
        try:
            tools = await mcp.list_tools()   # 通过其父类_MCPServerWithClientSession中的list_tools方法列出可用工具
            print(f"可用工具列表: {[tool.name for tool in tools]}")
        except Exception as e:
            print(f"获取工具列表失败: {e}")

&emsp;&emsp;在运行时，因为`Jupyter Notebook` 本身是基于事件循环的，所以可以使用`nest_asyncio` 库来解决这个问题。运行代码如下：

In [ ]:
import asyncio
import nest_asyncio

# 解决Jupyter Notebook 事件循环问题
nest_asyncio.apply()

# 运行主函数
asyncio.run(main())

连接MCP服务器成功: 12306_mcp
可用工具列表: ['get-current-date', 'get-stations-code-in-city', 'get-station-code-of-citys', 'get-station-code-by-names', 'get-station-by-telecode', 'get-tickets', 'get-interline-tickets', 'get-train-route-stations']


- **Step 7. 创建 Agent 实例**

&emsp;&emsp;最后创建 `Agent` 实例，并使用`mcp` 工具进行查询。这里仍然建议大家使用`python` 脚本运行，而不是在`Jupyter Notebook` 中运行。完整程序代码文件为`1_AgentSseMcp.py`，核心代码如下所示：

```python
        # step 1. 创建 MCP 服务器
        async with MCPServerSse(
            name="12306_mcp",
            params={
                "url": "https://mcp.api-inference.modelscope.cn/sse/56a8f2b1578f44",  # 这里需要替换成 MCP的 SSE URL
                "timeout": 30,            # HTTP请求超时时间30秒
                "sse_read_timeout": 5   # 连接超时时间到5分钟
            },
            client_session_timeout_seconds=60,  # 增加客户端会话超时到60秒
            cache_tools_list=True,             # 启用工具列表缓存
        ) as mcp:
            print(f"连接MCP服务器成功: {mcp.name}")  # 通过静态方法name查看当前连接的MCP服务器名称
            try:
                tools = await mcp.list_tools()   # 通过其父类_MCPServerWithClientSession中的list_tools方法列出可用工具
                print(f"可用工具列表: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"获取工具列表失败: {e}")
            
            # step 2. 创建 Agent 实例
            agent = Agent(               
                name="Train_Assistant",
                instructions="你是一个火车票查询助手，可以查询高铁信息",
                model=OpenAIChatCompletionsModel(
                    model=os.getenv("DEEPSEEK_MODEL"),
                    openai_client=deepseek_client,
                ),
                model_settings=ModelSettings(
                    temperature=0.6,
                    max_tokens=2048
                ),
                mcp_servers=[mcp],
            )

            # step 3. 运行 Agent 实例
            print(f"开始处理查询: {prompt}")
            result = await Runner.run(    
                starting_agent=agent,
                input=prompt,
                run_config=RunConfig(
                    tracing_disabled=True
                )
            )
            print(result.final_output)
```

&emsp;&emsp;运行结果如下图：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505201437922.png" width=60%></div>

&emsp;&emsp;除此以外，对接入多个`MCP`工具也并不复杂，只需要先创建MCP服务器的`URL`，使用异步上下文管理器同时连接并传递给`Agent`实例即可。这里我们在`ModelScope`中新创建了一个`飞常准`的`MCP` 服务，并生成了`SSE` 模式的`URL`，如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505201452697.png" width=60%></div>

&emsp;&emsp;完整程序代码文件为`2_AgentSseMcp_Multi.py`，运行后效果如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505201452820.png" width=60%></div>

&emsp;&emsp;综合实践来看，`SSE` 模式下的`MCP` 服务器接入相较于`Stdio` 模式确实更为简单，但也存在一个常见的问题就是：通过`平台托管平台`生成的`SSE` URL 并不会非常稳定。因为这种类型的服务基本都是在免费的云端资源上做的临时部署，无论是在稳定性，还是网络延迟上都会存在很大的问题，比较适用于快速构建`Agent` 原型，生产环境中还是建议以`自托管`的方式接入`MCP` 服务器，即下载`MCP` 服务器的源码，并基于`MCP`的`SSE`通信协议规范启动服务，从而提供一个稳定的`SSE` URL。

# 二、StreamableHttp MCP接入实战

&emsp;&emsp;`Streamable HTTP` 是 `MCP （Model Context Protocol）`在 2025-03-26 版规范中新引入的远程传输机制。它用「同一个 HTTP 端点同时支持 POST + GET」取代了旧的「POST + SSE 双端点」模式，使客户端和服务器可以在一次 POST 里把 JSON-RPC 请求发过去，服务器必要时再把连接“升级”为 SSE 流，把多条响应、进度或反向请求实时推送回来。这让远程 MCP 服务器不必保持长连接，天然适合无状态 / Serverless 部署，也能与旧 SSE 模式共存。

&emsp;&emsp;OpenAI Agent SDK 也在`v0.0.15`版本中支持了`Streamable HTTP` 模式，如下说明所示：https://github.com/openai/openai-agents-python/releases

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505201747237.png" width=60%></div>

&emsp;&emsp;同时提供了`MCPServerStreamableHttp`类来创建`Streamable HTTP` 模式的`MCP` 服务器。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505201749390.png" width=60%></div>

&emsp;&emsp;该模式可接收的配置参数定义与`MCPServerStdio`和`MCPServerSse`同样没有区别：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>MCPServerStreamableHttp可接收参数</font></p>
<div class="center">



| 参数名称                          | 类型                          | 默认值 | 描述                                                                                                                                                                                                 |
|-----------------------------------|-------------------------------|--------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `params`                          | `MCPServerStreamableHttpParams`        | -      | 配置服务器的参数，包括服务器的URL、请求头、HTTP请求超时和SSE连时。                                                               |
| `cache_tools_list`               | `bool`                        | `False`| 是否缓存工具列表。如果为 `True`，工具列表将被缓存，仅在第一次从服务器获取。如果为 `False`，每次调用 `list_tools()` 时都会从服务器获取工具列表。缓存可以通过调用 `invalidate_tools_cache()` 来失效。 |
| `name`                            | str 或 None               | `None` | 服务器的可读名称。如果未提供，将根据命令自动创建名称。                                                                                                                                              |
| `client_session_timeout_seconds`  | float 或 None                | `5`    | 传递给 MCP `ClientSession` 的读取超时时间时。                                                                                                                                                             |

&emsp;&emsp;区别在于`params`参数可接收的数据类型为`MCPServerStreamableHttpParams`，如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>MCPServerStreamableHttpParams参数汇总</font></p>
<div class="center">

| 参数                     | 类型               | 描述                                                         | 使用场景                                                   |
|--------------------------|--------------------|--------------------------------------------------------------|----------------------------------------------------------|
| `url`                    | `str`              | 服务器的 URL。                                             | 指定要连接的服务器地址。                                 |
| `headers`                | `dict` (可选)      | 发送到服务器的请求头。                                     | 用于传递认证信息或其他自定义头部。                       |
| `timeout`                | `float` (可选)     | HTTP 请求的超时时间，默认为 5 秒。                         | 控制请求的最大等待时间，避免长时间无响应。               |
| `terminate_on_close`  | `bool` (可选)          | 关闭时是否终止连接。                        | 在关闭连接时，决定是否立即终止与服务器的连接。
<div>                

&emsp;&emsp;但是这里有一点需要说明的是：虽然`OpenAI Agent SDK` 框架支持了`Streamable HTTP` 模式，但是因为是刚刚推出的新通信规范，目前公开的`MCP Server`支持`Streamable HTTP` 模式的并不多，甚至可以说几乎没有。像我们上面介绍的诸如`ModelScope`平台中MCP广场提供的`MCP Server`，其一键生成的`URL`也仅仅适用于通过`MCPServerSse`进行连接使用。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505211252964.png" width=60%></div>

&emsp;&emsp;因此，在`Streamable HTTP` 模式的`MCP Server`生态还不健全的情况下，目前常见的使用方式是基于`MCP SDK`自定义开发`MCP Server`，从而接入到`OpenAI Agent SDK` 框架进行使用。同样，关于如何使用`MCP Python SDK`开发`Streamable HTTP` 模式的`MCP Server`，大家可以在《【加餐】企业级流式HTTP MCP服务器开发实战》 进行学习。我们不再重复介绍。

&emsp;&emsp;这里我们直接把在《【加餐】企业级流式HTTP MCP服务器开发实战》中开发好的`MCP Server`拿过来做快速接入使用，即用于获取实时天气信息的`MCP-weather-http`项目，该项目源码已经上传到百度网盘中，大家可以直接下载后启动项目。如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505211304220.png" width=60%></div>

&emsp;&emsp;需要注意到的是：在启动的时候要通过 `--api-key` 参数指定一个`OpenWeather`的`API Key`用来获取天气信息。注册和获取`OpenWeather API Key`的地址为：https://openweathermap.org/api 。 启动后，会自动在本地启动一个`HTTP`服务，并提供一个`MCP`服务端点为：`http://localhost:3000`， 这就是我们需要在`MCPServerStreamableHttp`中配置的`URL`。

&emsp;&emsp;`OpenAI Agent SDK` 框架接入`Streamable HTTP` 模式的`MCP Server`的代码为：`3_AgentStreamableMcp.py`。在接入时，有以下几点需要重点关注：

1. 在本地启动的`HTTP`服务，在`MCPServerStreamableHttp`中需要通过`url`参数指定其`URL`为：`http://localhost:3000/mcp`， 即添加`/mcp`路径。
2. 一般标准协议下的`MCP Server`，在`headers`参数中需要添加`Accept`为：`text/event-stream, application/json`, 其中：
   - 初始握手/配置：通常使用 `application/json`
   - 事件流：使用 `text/event-stream`

&emsp;&emsp;服务器在会话生命周期的不同阶段可能需要使用不同的响应格式，否则的话就会出现 `406 Not Acceptable` 错误，一般都是 客户端的 `Accept` 头不支持服务器想要返回的格式时，就会触发该异常，同时这也是最容易出现的问题。大家需要根据实际的情况灵活调整。 运行效果如下所示：

- **服务端(mcp-weather-http)**

&emsp;&emsp;首先启动`mcp-weather-http`服务端，运行如下代码：

```bash
    python ./src/mcp_weather_http/server.py --api-key 01f0a3   # 注意这里替换成自己的 OpenWeather API Key
```


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505211319620.png" width=60%></div>

- **客户端（3_AgentStreamableMcp.py）**

&emsp;&emsp;运行`3_AgentStreamableMcp.py`，运行效果如下所示：

```bash
    python 3_AgentStreamableMcp.py
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505211319621.png" width=60%></div>

&emsp;&emsp;从运行效果中可以看到，`OpenAI Agent SDK` 框架成功连接到了`mcp-weather-http`服务端，并获取到了天气信息。同时在`mcp-weather-http`服务端中，会自动打印出`OpenAI Agent SDK` 框架的请求信息。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505211319622.png" width=60%></div>

&emsp;&emsp;至此，就已经成功的将`Streamable HTTP` 模式的`MCP Server`接入到了`OpenAI Agent SDK` 框架中，而正如之前所提到的，目前公开的`MCP Server`支持`Streamable HTTP` 模式的并不多，但因为`Streamable HTTP` 模式是最贴合企业级应用场景的通信模式，所以接下来的一段时间内一定会迅速普及，届时即可像使用`SSE`模式一样，直接在平台托管平台中生成`Streamable HTTP` 模式的`MCP Server`，然后直接在`OpenAI Agent SDK` 框架中使用。

&emsp;&emsp;至此，我们就完整介绍了`OpenAI Agents SDK` 框架中接入不同形式的`MCP Server`的方法，大家可以根据自己的实际开发现状灵活选择。  

# 三、Agents SDK 框架中的安全护栏


&emsp;&emsp;在 `OpenAI Agents SDK` 中，“Guardrails”（护栏）是一套以<font color=red>装饰器 + 钩子</font> 方式实现的输入/输出安全与质量控制框架。通过在`Agent`的生命周期关键节点（接收用户消息之前、生成最终回复之后）插入可并行或串行执行的校验函数，在发现风险时立即中断流程并抛出显式异常，从而为工程化落地提供“第一道也是最后一道”防线。正如我们在第一节课时介绍的`Agents SDK`的完整生命周期如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505091334117.png" width=60%></div>

&emsp;&emsp;`OpenAI Agents SDK` 框架简化版本的生命周期就是这样的：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505211413901.png" width=60%></div>

&emsp;&emsp;其中`Input Guardrails`为输入护栏，在实际运行`Runner`之前，会先执行`Input Guardrails`中的校验函数，如果校验失败，则直接中断流程或者返回手动定义的返回值。而`Output Guardrails`为输出护栏，在`Agent`生成最终回复之后，会执行`Output Guardrails`中的校验函数，如果校验失败，则不会正常输出`Agent`的回复。

&emsp;&emsp;本节课我们首先来看一下`Input Guardrails`在`Agents SDK` 框架中的具体实现。

&emsp;&emsp;`InputGuardrail` 是一个 `@dataclass` 泛型，保存 `guardrail_function、name` 等字段，并返回一个`GuardrailResult` 对象。其源码：https://github.com/openai/openai-agents-python/blob/main/src/agents/guardrail.py

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505211508648.png" width=60%></div>

&emsp;&emsp;输入护栏的核心组件如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>InputGuardrail 核心组件</font></p>
<div class="center">

| 概念                          | 描述                                                                                     |
|-------------------------------|------------------------------------------------------------------------------------------|
| **GuardrailFunctionOutput**   | 护栏函数的输出，包含结果信息和是否触发防护机制。                                          |
| **InputGuardrailResult**      | 包装输入护栏函数输出的结果对象，包含护栏和输出信息。                                      |
| **InputGuardrail**            | 输入护栏对象，包含护栏函数和名称，用于检查用户输入。                                      |
| **@input_guardrail**          | 装饰器，用于将普通函数转换为输入护栏函数，便于创建和使用护栏。                            |

&emsp;&emsp;这套源码组件的实现可以明确说明在`OpenAI Agents SDK` 框架中对`InputGuardrail` 的使用方法。首先，在构建安全护栏时，`@input_guardrail()` 直接将函数声明为输入护栏函数，也可以通过显式实例化 `InputGuardrail` 对象来创建输入护栏对象，然后，这个护栏对象会返回` GuardrailFunctionOutput` 对象，包含`output_info`(护栏检查结果的输出信息) 和 `tripwire_triggered` 字段，用于判断是否触发防护机制。最终，护栏函数执行后，返回`InputGuardrailResult` 对象，包含`guardrail`(输入护栏对象) 和 `output`(护栏函数输出) 字段。其完整生命周期如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505211528592.png" width=60%></div>

&emsp;&emsp;在`Runner`运行时，`InputGuardrail` 的校验函数的工作原理如下所示：

1. **Runner 收集护栏**：当调用 `Runner.run_*` 时，它会从 `agent.input_guardrails` 里拉出全部输入护栏。
2. **并发启动**：`Runner` 在 `_run_input_guardrails` 阶段把所有护栏函数以 `asyncio.gather` 并行运行，主代理也开始推理；谁先触发 `tripwire`，谁就中断整条链路。
3. **Tripwire 处理（中断）**：护栏返回 GuardrailResult.tripwire("reason") → Runner 立刻抛 InputGuardrailTripwireTriggered，并把异常冒泡到上层或 HTTP 4xx;

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505221028211.png" width=60%></div>

&emsp;&emsp;源码位置：https://github.com/openai/openai-agents-python/blob/main/src/agents/run.py#L182 , 感兴趣的同学可以自行阅读源码。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505211542541.png" width=60%></div>

&emsp;&emsp;理解了以上原理后，我们便可以开始基于`InputGuardrail` 进行安全护栏的构建了。接下来我们会依次尝试使用装饰器 `@input_guardrail()` 和显式实例化 `InputGuardrail` 对象来创建输入护栏。通过实际的示例来看一下这两种不同方式如何使用，以及各自适用于什么应用开发场景。

&emsp;&emsp;首先需要明确的是，构建输入护栏需要在创建`Agent`对象时通过`input_guardrails`参数进行传递：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Agent 组件核心参数</font></p>
<div class="center">

| 属性名                | 类型                                                                                          | 描述                                                                                                   |
|---------------------|-----------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------|
| <font color=green>**name**</font>             | `str`                                                                                         | 代理的名称。                                                                                             |
| <font color=green>**instructions**</font>      | `str` \| `Callable[[RunContextWrapper[TContext], Agent[TContext]], MaybeAwaitable[str]]` \| `None` | 代理的指令，用作“系统提示”。可以是字符串或动态生成指令的函数。                                           |
| <font color=green>**handoff_description**</font> | `str` \| `None`                                                                              | 代理的描述，用于代理作为交接时，让 LLM 知道它的功能和何时调用它。                                         |
| <font color=green>**handoffs**</font>         | list[Agent[Any] | Handoff[TContext]]                                                   | 代理可以委托的子代理列表。允许关注点分离和模块化。                                                       |
| <font color=green>**model**</font>              | `str` \| `Model` \| `None`                                                                   | 调用 LLM 时使用的模型实现。默认情况下，如果未设置，代理将使用 `openai_provider.DEFAULT_MODEL` 中配置的默认模型。 |
| <font color=green>**model_settings**</font>    | `ModelSettings`                                                                               | 配置模型特定的调优参数（例如温度、top_p）。                                                             |
| <font color=green>**tools**</font>       | `list[Tool]`                                                                                 | 代理可以使用的工具列表。                                                                                 |
| <font color=green>mcp_servers</font>   | `list[MCPServer]`                                                                             | 代理可以使用的模型上下文协议（MCP）服务器列表。                                                        |
| <font color=green>mcp_config</font>        | `MCPConfig`                                                                                   | MCP 服务器的配置。                                                                                       |
| <font color=red>input_guardrails</font>  | `list[InputGuardrail[TContext]]`                                                             | 在代理执行之前并行运行的检查列表，仅在代理是链中的第一个代理时运行。                                     |
| `output_guardrails` | `list[OutputGuardrail[TContext]]`                                                            | 在生成响应后对代理的最终输出运行的检查列表，仅在代理生成最终输出时运行。                                 |
| `output_type`       | type[Any] | AgentOutputSchemaBase | None                                                 | 输出对象的类型。如果未提供，输出将为 `str`。                                                             |
| <font color=green>hooks</font>             | AgentHooks[TContext] | None                                                               | 接收代理生命周期事件回调的类。                                                                           |
| `tool_use_behavior` | Literal["run_llm_again", "stop_on_first_tool"] | StopAtTools | ToolsToFinalOutputFunction | 配置工具使用的处理方式。                                                                                 |
| `reset_tool_choice` | `bool`                                                                                       | 调用工具后是否将工具选择重置为默认值。默认为 `True`。确保代理不会进入工具使用的无限循环。                   |

</div>

- **使用装饰器 `@input_guardrail()` 直接将函数声明为输入护栏函数**

&emsp;&emsp;安全护栏组件无论是输入护栏还是输出护栏，都需要结合具体的应用场景才有意义。对于输入护栏来说，其核心本质上就是对用户输入的校验，从而确保输入的安全性和合法性。这里我们以`1_AgentSseMcp.py`中接入`12306`购票系统查询火车、高铁的`Agent`为例，来构建一个输入护栏，具有实际意义的可以包含如下几种类型：

1. 不当言论过滤：防止用户使用侮辱性或不适当的语言
2. 非交通相关查询过滤：防止用户查询与火车、高铁无关的内容
3. 身份证信息泄露过滤：防止用户查询时泄露身份证信息
4. 信息安全护栏：防止用户回去敏感信息或系统信息 

&emsp;&emsp;完整的代码在`4_inputGuardrail.py`中，这里我们需要重点介绍一下构建输入护栏时注意的点：

&emsp;&emsp;1. 护栏函数对象需要在构建`Agent`对象时作为`input_guardrails`参数传入，可以通过列表传递多个护栏函数对象，其运行过程是并行的，其伪代码如下：

```python
    agent = Agent(
        name="12306购票系统查询",
        instructions="只回答12306购票系统查询相关问题",
        input_guardrails=[guardrail_function1, guardrail_function2, guardrail_function3]  # 可以传递多个护栏函数对象
    )
```


&emsp;&emsp;当传递多个护栏对象时，只要触发了任意一个护栏函数的`tripwire`，就会立即中断`Agent`的运行。源码位置： https://github.com/openai/openai-agents-python/blob/main/src/agents/run.py#L182

```python
    for done in asyncio.as_completed(guardrail_tasks):
        result = await done
        if result.output.tripwire_triggered:
            # 取消所有护栏任务并引发异常
            for t in guardrail_tasks:
                t.cancel()
            raise InputGuardrailTripwireTriggered(result)
```


2. 当使用`@input_guardrail`装饰器构建输入护栏函数对象时，系统会将护栏函数包装在`InputGuardrail`类中，该类的`run`方法会调用你的函数并传入这三个参数。源码位置：https://github.com/openai/openai-agents-python/blob/main/src/agents/guardrail.py

```python
    async def run(
        self,
        agent: Agent[Any],
        input: str | list[TResponseInputItem],
        context: RunContextWrapper[TContext],
    ) -> InputGuardrailResult:
        if not callable(self.guardrail_function):
            raise UserError(f"Guardrail function must be callable, got {self.guardrail_function}")

        output = self.guardrail_function(context, agent, input)
        if inspect.isawaitable(output):
            return InputGuardrailResult(
                guardrail=self,
                output=await output,
            )

        return InputGuardrailResult(
            guardrail=self,
            output=output,
        )
```

&emsp;&emsp;所以标准的写法是在函数签名中传入这三个参数，这三个参数分别表示上下文、Agent对象和用户输入。即使不在函数体内使用这些参数，也必须在函数签名中声明它们，否则`Python`会抛出参数不匹配的`TypeError`异常。

```python
    @input_guardrail
    async def non_transportation_query_guardrail(ctx: RunContextWrapper[None], agent, input_text: str):
        pass
```

&emsp;&emsp;3. 在输入护栏中灵活应用`GuardrailFunctionOutput`对象，其中`output_info`字段可以包含任意信息，你可以把任意想要经过安全护栏校验的信息放在自定义的字段中，而`tripwire_triggered`字段用于判断是否触发防护机制，这是一个布尔值，通过控制这个值的返回去决定是否触发防护机制。源码位置：https://github.com/openai/openai-agents-python/blob/main/src/agents/guardrail.py

```python
    @dataclass
    class GuardrailFunctionOutput:
        """The output of a guardrail function."""

        output_info: Any
        """
        Optional information about the guardrail's output. For example, the guardrail could include
        information about the checks it performed and granular results.
        """

        tripwire_triggered: bool
        """
        Whether the tripwire was triggered. If triggered, the agent's execution will be halted.
        """
```


&emsp;&emsp;运行效果如下所示，大家可以自行修改和尝试：

```bash
    python 4_inputGuardrail.py
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505211944433.png" width=60%></div>

&emsp;&emsp;使用装饰器 `@input_guardrail()` 创建输入护栏实例非常适用于简单的单一护栏逻辑，比如每个护栏的逻辑都是独立的，不需要参数化，在开发时即可确定同时在运行时不需要进行更改。而当需要更复杂和灵活的护栏逻辑时，则需要使用显式实例化 `InputGuardrail` 对象的方式来创建护栏实例。

- **显式实例化 `InputGuardrail` 对象创建护栏实现运行时的热插拔**

&emsp;&emsp;从源码可以看到，`@input_guardrail`装饰器最终也只是创建了`InputGuardrail`的实例，其实内部就是在做如下转化： 源码位置：https://github.com/openai/openai-agents-python/blob/main/src/agents/guardrail.py

```python
    def input_guardrail(...):
        def decorator(f):
            return InputGuardrail(guardrail_function=f, name=name)
        # ...
```

&emsp;&emsp;所以我们可以直接显式实例化`InputGuardrail`对象来创建护栏实例，并将其添加到`Agent`对象的`input_guardrails`列表中。如下代码所示：而这种方式最大的优势在于可以将护栏逻辑与护栏实例创建分离，从而实现运行时的热插拔。比如在对用户输入的问题是否违规时，可以灵活的定义不同的违规规则。这是非常适用于实际的开发场景的。

&emsp;&emsp;这里大家可以看下`5_inputGuardrailDynamic.py`中的代码，`check_inappropriate_language`等函数的内部逻辑与`4_inputGuardrail.py`中的代码是完全一致的，只是去掉了装饰器。然后使用工厂方法创建了护栏实例，可以直接在初始化的时候，不修改护栏内部的逻辑构建自定义的参数，从而实现运行时的热插拔。

&emsp;&emsp;运行效果如下所示：

```bash
    python 5_inputGuardrailDynamic.py
```


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505212013956.png" width=100%></div>

&emsp;&emsp;这种显式实例化让你可以把`custom_topic_guardrail`存在配置中心、动态增删，甚至按租户灰度开关等场景下。比如：


```python
    # 从配置中心读取护栏配置
    guardrail_configs = config_center.get_guardrails_config(tenant_id="tenant1")

    # 动态创建护栏列表
    input_guardrails = []
    for config in guardrail_configs:
        if config["enabled"]:
            guardrail = create_keyword_guardrail(
                name=config["name"],
                keywords=config["keywords"],
                description=config["description"],
                trigger_condition=lambda kw: len(kw) > config["threshold"]
            )
            input_guardrails.append(guardrail)

    # 按租户ID获取适用的护栏
    def get_tenant_guardrails(tenant_id):
        # 获取租户基本配置
        tenant_config = db.get_tenant_config(tenant_id)
        
        # 基础护栏对所有租户生效
        guardrails = [inappropriate_language_guardrail]
        
        # VIP租户启用高级护栏
        if tenant_config.tier == "VIP":
            guardrails.append(system_security_guardrail)
            
        # 特定行业租户启用行业特定护栏
        if tenant_config.industry == "finance":
            guardrails.append(create_keyword_guardrail(
                name="finance_compliance",
                keywords=db.get_finance_keywords(),
                description="金融合规检查",
                trigger_condition=lambda kw: len(kw) > 0
            ))
        
        return guardrails
```

&emsp;&emsp;对于复杂的企业应用，特别是`SaaS`平台，这种灵活性是非常适用的，可以让产品团队更快地迭代和优化安全策略，也让运营团队能够根据实际情况动态调整护栏规则。

- **外部 Agent 做输入护栏**

&emsp;&emsp;最常见的一种方式是把轻量模型封装成独立`Agent`，作为输入护栏挂载，从而实现复杂意图识别、合规审查等需求。 这里可以运行`6_inputGuardrailAgent.py`文件。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505221050654.png" width=60%></div>